In [ ]:
import pyvacon
import pyvacon.marketdata.testdata as mkt_testdata
import pyvacon.tools.enums as enums
import pyvacon.marketdata.plot as mkt_plot
import pyvacon.models.plot as model_plot
import pyvacon.models.tools as model_tools
import pyvacon.analytics as analytics
import pyvacon.tools.converter as converter

import datetime as dt
import math
import numpy as np

import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

import ipywidgets as widgets

In [ ]:
# We define some constants, which we'll use repeatedly throughout this notebook
grid_alpha = 0.4
daycounter_type_standard = enums.DayCounter.ACTACT
interpolation_type_standard = enums.InterpolationType.LINEAR
extrapolation_type_standard = enums.ExtrapolationType.NONE
plotly_scatter_mode_standard = 'lines'

# Interest Rate Shock Scenarios

In [ ]:
# Define functions to help compute the shock scenarios

def shortRateShock(t, basispoints):
    if isinstance(t, (list, np.ndarray)):
        result = []
        for x in t:
            result.append(shortRateShock(x, basispoints))
        #print('list')
        return result
    #print('scalar')
    #print(type(t))
    return np.exp(-t/4) * basispoints


def longRateShock(t, basispoints):
    if isinstance(t, (list, np.ndarray)):
        result = []
        for x in t:
            result.append(longRateShock(x, basispoints))
        return result
    return (1-math.exp(-t/4)) * basispoints


def steepener(t, basispointsShort, basispointsLong):
    if isinstance(t, (list, np.ndarray)):
        result = []
        for x in t:
            result.append(steepener(x, basispointsShort, basispointsLong))
        return result
    return -0.65 * shortRateShock(t, basispointsShort)  +  0.9 * longRateShock(t, basispointsLong)


def flattener(t, basispointsShort, basispointsLong):
    if isinstance(t, (list, np.ndarray)):
        result = []
        for x in t:
            result.append(flattener(x, basispointsShort, basispointsLong))
        return result
    return 0.8 * shortRateShock(t, basispointsShort)  -  0.6 * longRateShock(t, basispointsLong)


def lowerBound(t):
    if isinstance(t, (list, np.ndarray)):
        result = []
        for x in t:
            result.append(lowerBound(x))
        return result
    return min(0, -100 + 5*t) # in basispoints


def getShockValue(t, shockScenario, parallel = 0, short = 0, long = 0):
    
    if shockScenario == '':
        return 0
    
    if shockScenario == 'Original':
        return 0
    
    
    
    if shockScenario == 'Parallel':
        return parallel
    
    if shockScenario == 'ParallelUp':
        return parallel
    
    if shockScenario == 'ParallelDown':
        return -parallel
    
    
    
    if shockScenario == 'Short':
        return shortRateShock(t, short)
    
    if shockScenario == 'ShortUp':
        return shortRateShock(t, short)
    
    if shockScenario == 'ShortDown':
        return shortRateShock(t, -short)
    
    
    
    if shockScenario == 'Long':
        return longRateShock(t, long)
    
    if shockScenario == 'LongUp':
        return longRateShock(t, long)
    
    if shockScenario == 'LongDown':
        return longRateShock(t, -long)
    
    
    
    if shockScenario == 'Flatten':
        return flattener(t, short, long)
    
    if shockScenario == 'Steepen':
        return steepener(t, short, long)
    
    
    raise Exception('I don\'t know a scenario of the name \'' + shockScenario + '\'')

In [ ]:
# Define a function, which shifts a vector of interest rates according to a given shock scenario

def getShockedInterestRates(
        refdate,
        dates,
        interestRates,
        daycounter,
        shockScenario,
        parallel = 0,
        short = 0,
        long = 0,
        useLowerBound = True):
    if len(interestRates) != len(dates):
        raise Exception('You need to provide an equal number of interest rates and sampling dates.')
    
    shockedInterestRates = []
    
    for i in range(len(dates)):
        t = daycounter.yf(refdate, dates[i])
        rate = interestRates[i] + getShockValue(t, shockScenario, parallel, short, long)
        if useLowerBound:
            rate = max(rate, lowerBound(t))
        shockedInterestRates.append(rate)
    
    #print(shockedInterestRates)
    
    return shockedInterestRates

In [ ]:
# Define a function, which shifts a vector of discount factors according to a given shock scenario

def getShockedDiscountFactors(
        refdate,
        dates,
        interestRates,
        daycounter,
        shockScenario,
        parallel = 0,
        short = 0,
        long = 0,
        useLowerBound = True):
    
    if len(interestRates) != len(dates):
        raise Exception('You need to provide an equal number of discount factors and sampling dates.')
    
    shockedInteresRates = getShockedInterestRates(
        refdate,
        dates,
        interestRates,
        daycounter,
        shockScenario,
        parallel,
        short,
        long,
        useLowerBound
    )    
    
    shockedDiscountFactors = []
    #shockedDiscountFactors = analytics.vectorDouble(len(dates))
    #print(shockedDiscountFactors)
    
    for i in range(len(dates)):
        t = daycounter.yf(refdate, dates[i])
        rate = shockedInteresRates[i]/100/100 # are given in basis points -> convert to decimal number
        shockedDiscountFactors.append(math.exp(-t*rate))
        #shockedDiscountFactors[i] = discountFactors[i] + getShockValue(t, shockScenario, parallel, short, long)
        #print(shockedDiscountFactors)
    
    #print("//////////////////////////////////////////////")
    #print(shockedInteresRates)
    #print("-----------------------")
    #print(shockedDiscountFactors)
    #print("//////////////////////////////////////////////")
    
    return shockedDiscountFactors

In [ ]:
# Define a function, which shifts a discount curve according to a given shock scenario   
# We now use these shifted discount factors to construct shifted discount curves     
    
def getShockedDiscountCurve(
        name,
        refdate,
        dates,
        interestRates,
        daycounterType,
        interpolationType,
        extrapolationType,
        shockScenario,
        parallel = 0,
        short = 0,
        long = 0,
        useLowerBound = True):
    
    shockedDFs = getShockedDiscountFactors(
        refdate,
        dates,
        interestRates,
        analytics.DayCounter(daycounterType),
        shockScenario,
        parallel,
        short,
        long,
        useLowerBound
    )    
    
    #print(shockedDFs)
    
    return analytics.DiscountCurve(
        name,
        refdate,
        dates,
        shockedDFs,
        daycounterType,
        interpolationType,
        extrapolationType
    )  

## The scenarios

In [ ]:
# Define the parameters for the shock scenarios
shockparams_by_currency = pd.DataFrame({'Currency': [], 'Parallel': [], 'Short': [], 'Long': []})
shockparams_by_currency = shockparams_by_currency.append({'Currency': 'EUR', 'Parallel': 200, 'Short': 250, 'Long': 100}, ignore_index = True)
shockparams_by_currency = shockparams_by_currency.append({'Currency': 'GBP', 'Parallel': 250, 'Short': 300, 'Long': 150}, ignore_index = True)
shockparams_by_currency = shockparams_by_currency.append({'Currency': 'USD', 'Parallel': 200, 'Short': 300, 'Long': 150}, ignore_index = True)

# Choose the standard params we're using in this notebook
shockparams_standard = shockparams_by_currency.loc[shockparams_by_currency['Currency'] == 'EUR'].loc[0]

### Plot the shock scenarios
In this section, we illustrate the interest rate scenarios by applying them to an interest rate curve with constant interest rate 0.

In [ ]:
refdate_scenarios = dt.datetime(year = 2020, month = 1 , day = 1)

sampling_points_simple_example_yf = [1/365] # 1 day
sampling_points_simple_example_yf.extend( (np.arange(11)+1)/12 ) # 1 to 11 months
sampling_points_simple_example_yf.extend(np.arange(30)+1) # 1 to 30 years

sampling_points_simple_example = []
sampling_points_simple_example.append(refdate_scenarios + dt.timedelta(days = 1))
for i in range(11):
    sampling_points_simple_example.append(refdate_scenarios + dt.timedelta(days = (i+1)*30))
for i in range(30):
    year = refdate_scenarios.year + i + 1
    month = refdate_scenarios.month
    day = refdate_scenarios.day
    sampling_points_simple_example.append(
        dt.datetime(year = year, month = month , day = day)
    )

In [ ]:
# def plot_shock_scenarios(
#     samplingPoints,
#     shockParams
# ):
#     parallel = shockParams['Parallel']
#     short = shockParams['Short']
#     long = shockParams['Long']   
    
#     fig = go.Figure()

#     fig.add_trace(go.Scatter(x = samplingPoints, y = [0]*len(samplingPoints), name = 'Original', mode = plotly_scatter_mode_standard))
#     fig.add_trace(go.Scatter(x = samplingPoints, y = [parallel]*len(samplingPoints), name = 'ParallelUp', mode = plotly_scatter_mode_standard))
#     fig.add_trace(go.Scatter(x = samplingPoints, y = [-parallel]*len(samplingPoints), name = 'ParallelDown', mode = plotly_scatter_mode_standard))
#     fig.add_trace(go.Scatter(x = samplingPoints, y = shortRateShock(samplingPoints, short), name = 'ShortUp', mode = plotly_scatter_mode_standard))
#     fig.add_trace(go.Scatter(x = samplingPoints, y = shortRateShock(samplingPoints, -short), name = 'ShortDown', mode = plotly_scatter_mode_standard))
#     fig.add_trace(go.Scatter(x = samplingPoints, y = longRateShock(samplingPoints, long), name = 'LongUp', mode = plotly_scatter_mode_standard))
#     fig.add_trace(go.Scatter(x = samplingPoints, y = longRateShock(samplingPoints, -long), name = 'LongDown', mode = plotly_scatter_mode_standard))
#     fig.add_trace(go.Scatter(x = samplingPoints, y = flattener(samplingPoints, short, long), name = 'Flattener', mode = plotly_scatter_mode_standard))
#     fig.add_trace(go.Scatter(x = samplingPoints, y = steepener(samplingPoints, short, long), name = 'Steepener', mode = plotly_scatter_mode_standard))

#     fig.update_layout(
#         showlegend=True,
#         xaxis = dict(title_text = "Expiry (in years)"),
#         yaxis = dict(title_text = "Interest rate (in basis points)")
#     )

#     fig.show()

    

# plot_shock_scenarios(sampling_points_simple_example_yf, shockparams_standard)

In [ ]:
# Plot the shock scenarios (once with and once without enforcing the lower bound for interest rates)
def plot_shocked_interest_rates(
    refdate,
    dates,
    interestRates,
    daycounter,
    shockParams,
    useLowerBound = True
):
    parallel = shockParams['Parallel']
    short = shockParams['Short']
    long = shockParams['Long']
    
    shockScenarios = ['Original', 'ParallelUp', 'ParallelDown', 'ShortUp', 'ShortDown', 'LongUp', 'LongDown', 'Flatten', 'Steepen']
    
    
    fig = go.Figure()
    
    for shockScenario in shockScenarios:
        ir = getShockedInterestRates(
            refdate,
            dates,
            interestRates,
            daycounter,
            shockScenario,
            parallel,
            short,
            long,
            useLowerBound
        )
        year_fractions = []
        for i in range(len(dates)):
            year_fractions.append(daycounter.yf(refdate, dates[i]))
        
        fig.add_trace(go.Scatter(x = year_fractions, y = ir, name = shockScenario, mode = plotly_scatter_mode_standard))
     
    if  useLowerBound:
        title_figure = 'Shock scenarios (with lower bound)'
    else :
        title_figure = 'Shock scenarios (without lower bound)'
    
    fig.update_layout(
        showlegend=True,
        xaxis = dict(title_text = "Expiry (in years)"),
        yaxis = dict(title_text = "Interest rate (in basis points)"),
        title={
            'text': title_figure,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'}
    )

    fig.show()

interestRates = [0] * len(sampling_points_simple_example)

plot_shocked_interest_rates(
    refdate_scenarios,
    sampling_points_simple_example,
    interestRates,
    analytics.DayCounter(daycounter_type_standard),
    shockparams_standard,
    False
)

plot_shocked_interest_rates(
    refdate_scenarios,
    sampling_points_simple_example,
    interestRates,
    analytics.DayCounter(daycounter_type_standard),
    shockparams_standard,
    True
)

### Plot the shock scenarios (discount factors)

In [ ]:
# Now, plot the discount factors instead of the interest rates for each scenario
def plot_shocked_discount_factors(
    refdate,
    dates,
    interestRates,
    daycounter,
    shockParams,
    useLowerBound = True
):
    parallel = shockParams['Parallel']
    short = shockParams['Short']
    long = shockParams['Long']
    
    shockScenarios = ['Original', 'ParallelUp', 'ParallelDown', 'ShortUp', 'ShortDown', 'LongUp', 'LongDown', 'Flatten', 'Steepen']
    
    
    fig = go.Figure()
    
    for shockScenario in shockScenarios:
        df = getShockedDiscountFactors(
            refdate,
            dates,
            interestRates,
            daycounter,
            shockScenario,
            parallel,
            short,
            long,
            useLowerBound
        )
        year_fractions = []
        for i in range(len(dates)):
            year_fractions.append(daycounter.yf(refdate, dates[i]))
        
        fig.add_trace(go.Scatter(x = year_fractions, y = df, name = shockScenario, mode = plotly_scatter_mode_standard))
     
    if  useLowerBound:
        title_figure = 'Shock scenarios (with lower bound)'
    else :
        title_figure = 'Shock scenarios (without lower bound)'
    
    fig.update_layout(
        showlegend=True,
        xaxis = dict(title_text = "Expiry (in years)"),
        yaxis = dict(title_text = "Discount factor"),
        title={
            'text': title_figure,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'}
    )

    fig.show()


interestRates = [0] * len(sampling_points_simple_example)

plot_shocked_discount_factors(
    refdate_scenarios,
    sampling_points_simple_example,
    interestRates,
    analytics.DayCounter(daycounter_type_standard),
    shockparams_standard,
    False
)

plot_shocked_discount_factors(
    refdate_scenarios,
    sampling_points_simple_example,
    interestRates,
    analytics.DayCounter(daycounter_type_standard),
    shockparams_standard,
    True
)

#### Plot them again, using discount curves

In [ ]:
# Plot the discount factors again, but use pyvacon discount curves instead of mere discount factors for the computations
def plot_shocked_discount_curves(
    name,
    refdate,
    dates,
    discountFactors,
    daycounterType,
    interpolationType,
    extrapolationType,
    shockParams,
    useLowerBound = True
):
    parallel = shockParams['Parallel']
    short = shockParams['Short']
    long = shockParams['Long']
    
    shockScenarios = ['Original', 'ParallelUp', 'ParallelDown', 'ShortUp', 'ShortDown', 'LongUp', 'LongDown', 'Flatten', 'Steepen']
   
    daycounter = analytics.DayCounter(daycounterType)
    
    fig = go.Figure()
    
    for shockScenario in shockScenarios:
        dc = getShockedDiscountCurve(
            name + '_' + shockScenario,
            refdate,
            dates,
            discountFactors,
            daycounterType,
            interpolationType,
            extrapolationType,
            shockScenario,
            parallel,
            short,
            long,
            useLowerBound
        )
        
        year_fractions = []
        for i in range(len(dates)):
            year_fractions.append(daycounter.yf(refdate, dates[i]))
        values = analytics.vectorDouble()
        dc.value(values, refdate, dates)
        
        # convert to normal list
        values = [x for x in values]
            
        fig.add_trace(go.Scatter(x = year_fractions, y = values, name = shockScenario, mode = plotly_scatter_mode_standard))
     
    
    if  useLowerBound:
        title_figure = 'Shock scenarios (with lower bound)'
    else :
        title_figure = 'Shock scenarios (without lower bound)'
    
    fig.update_layout(
        showlegend=True,
        xaxis = dict(title_text = "Expiry (in years)"),
        yaxis = dict(title_text = "Discount factor"),
        title={
            'text': title_figure,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'}
    )

    fig.show()
    
    
plot_shocked_discount_curves(
    name = 'dc_linear',
    refdate = refdate_scenarios,
    dates = sampling_points_simple_example,
    discountFactors = [0.0] * len(sampling_points_simple_example),
    daycounterType = daycounter_type_standard,
    interpolationType = interpolation_type_standard,
    extrapolationType = extrapolation_type_standard,
    shockParams = shockparams_standard,
    useLowerBound = False
)    
    
plot_shocked_discount_curves(
    name = 'dc_linear',
    refdate = refdate_scenarios,
    dates = sampling_points_simple_example,
    discountFactors = [0.0] * len(sampling_points_simple_example),
    daycounterType = daycounter_type_standard,
    interpolationType = interpolation_type_standard,
    extrapolationType = extrapolation_type_standard,
    shockParams = shockparams_standard,
    useLowerBound = True
)

## Example (BaFin)
In this section, we apply the interest rate scenarios to a more realistic interest rate curve. The curve we use is taken from *Rundschreiben 06/2019 (BA) - Zinsänderungsrisiken im Anlagenbuch* an official regulatory document issued by BaFin.

In [ ]:
# Define example data

refdate_BaFin = dt.datetime(year = 2020, month = 1 , day = 1)

example_BaFin_interest_rate_basispoints = [
    -57,
    -60,
    -62,
    -64,
    -69,
    -68,
    -64,
    -56,
    -48,
    -39,
    -30,
    -21,
    -13,
    -6,
    1,
    7,
    13,
    18,
    22,
    26,
    30,
    33,
    36,
    38,
    41,
    43,
    45,
    47,
    49,
    50,
    52,
    53,
    55
]

example_BaFin_sampling_points = [
    refdate_BaFin + dt.timedelta(days = round(yf*365)) for yf in [0.25, 0.5, 0.75]
]

for i in range(30):
    year = refdate_BaFin.year + i + 1
    month = refdate_BaFin.month
    day = refdate_BaFin.day
    example_BaFin_sampling_points.append(
        dt.datetime(year = year, month = month , day = day)
    )

### Plot all scenarios

In [ ]:
# Plot the scenarios using a function defined above
plot_shocked_interest_rates(
    refdate_BaFin,
    example_BaFin_sampling_points,
    example_BaFin_interest_rate_basispoints,
    analytics.DayCounter(daycounter_type_standard),
    shockparams_standard,
    False
)

plot_shocked_interest_rates(
    refdate_BaFin,
    example_BaFin_sampling_points,
    example_BaFin_interest_rate_basispoints,
    analytics.DayCounter(daycounter_type_standard),
    shockparams_standard,
    True
)

## Example (current market data)

In [ ]:
# Define the example data
example_current_interest_rates = [
    -0.45,
    -0.423,
    -0.3925,
    -0.363,
    -0.36517,
    -0.36636,
    -0.37028,
    -0.37527,
    -0.37887,
    -0.38303,
    -0.38643,
    -0.38914,
    -0.39686,
    -0.396,
    -0.379,
    -0.342,
    -0.298,
    -0.252,
    -0.203,
    -0.154,
    -0.107,
    -0.057,
    -0.01,
    0.036,
    0.07833,
    0.11403,
    0.145,
    0.16381,
    0.18032,
    0.19499,
    0.20824,
    0.22,
    0.213,
    0.178
    #,0.054
]

example_current_interest_rates_basispoints = [100*r for r in example_current_interest_rates]

refdate_current = dt.datetime(year = 2020, month = 3 , day = 31)

example_current_sampling_points = []
example_current_sampling_points.append(refdate_current + dt.timedelta(days = 1))
for i in range(11):
    example_current_sampling_points.append(refdate_current + dt.timedelta(days = (i+1)*30))
for i in range(20):
    year = refdate_current.year + i + 1
    month = refdate_current.month
    day = refdate_current.day
    example_current_sampling_points.append(
        dt.datetime(year = year, month = month , day = day)
    )
for i in [25, 30]:#, 50]:
    year = refdate_current.year + i
    month = refdate_current.month
    day = refdate_current.day
    example_current_sampling_points.append(
        dt.datetime(year = year, month = month , day = day)
    )
    

### Plot all scenarios

In [ ]:
# Use a function defined above to plot the scenarios
plot_shocked_interest_rates(
    refdate_current,
    example_current_sampling_points,
    example_current_interest_rates_basispoints,
    analytics.DayCounter(daycounter_type_standard),
    shockparams_standard,
    False
)

plot_shocked_interest_rates(
    refdate_current,
    example_current_sampling_points,
    example_current_interest_rates_basispoints,
    analytics.DayCounter(daycounter_type_standard),
    shockparams_standard,
    True
)

## Interactive Plot

In [ ]:
# Define a function, which applies and then plots a single scenario

def plot_shocked_interest_rates_single(
    name,
    fig,
    showPlot,
    refdate,
    dates,
    interestRates,
    daycounter,
    shockScenario,
    shockParams,
    useLowerBound = True,
    plotOriginal = True,
    plotLowerBound = True
):
    parallel = shockParams['Parallel']
    short = shockParams['Short']
    long = shockParams['Long']
    
    
    year_fractions = []
    for i in range(len(dates)):
        year_fractions.append(daycounter.yf(refdate, dates[i]))
    
    if showPlot:
        visible = True
    else:
        visible = 'legendonly'
        
    if plotOriginal:
        fig.add_trace(go.Scatter(x = year_fractions, y = interestRates, name = name + ' (Original)', mode = plotly_scatter_mode_standard, visible = visible))

        
    # plot chosen scenario  
    if shockScenario != '' and shockScenario != 'Original':
        ir = getShockedInterestRates(
            refdate,
            dates,
            interestRates,
            daycounter,
            shockScenario,
            parallel,
            short,
            long,
            useLowerBound
        )
        fig.add_trace(go.Scatter(x = year_fractions, y = ir, name = name + ' (' + shockScenario + ')', mode = plotly_scatter_mode_standard, visible = visible))

    
    return year_fractions;

In [ ]:
# Define the function, which we're going to use in combination with widgets in order to plot
# the scenario specified by the user.

def showPlot(
    shockScenario = 'Original',
    useLowerBound = True
):
    yf_bafin = []
    yf_current = []
    yf_constant = []
    
    showExampleConstant = False
    showExampleBaFin = True
    showExampleCurrent = False
    
    fig = go.Figure()
    
    yf_constant = plot_shocked_interest_rates_single(
        'Constant 0',
        fig,
        showExampleConstant,
        refdate = refdate_scenarios,
        dates = sampling_points_simple_example,
        interestRates = interestRates,
        daycounter = analytics.DayCounter(daycounter_type_standard),
        shockScenario = shockScenario,
        shockParams = shockparams_standard,
        useLowerBound = useLowerBound,
        plotOriginal = True,
        plotLowerBound = useLowerBound
    )

    yf_bafin = plot_shocked_interest_rates_single(
        'BaFin',
        fig,
        showExampleBaFin,
        refdate = refdate_BaFin,
        dates = example_BaFin_sampling_points,
        interestRates = example_BaFin_interest_rate_basispoints,
        daycounter = analytics.DayCounter(daycounter_type_standard),
        shockScenario = shockScenario,
        shockParams = shockparams_standard,
        useLowerBound = useLowerBound,
        plotOriginal = True,
        plotLowerBound = useLowerBound
    )

    yf_current = plot_shocked_interest_rates_single(
        'Current',
        fig,
        showExampleCurrent,
        refdate = refdate_current,
        dates = example_current_sampling_points,
        interestRates = example_current_interest_rates_basispoints,
        daycounter = analytics.DayCounter(daycounter_type_standard),
        shockScenario = shockScenario,
        shockParams = shockparams_standard,
        useLowerBound = useLowerBound,
        plotOriginal = True,
        plotLowerBound = useLowerBound
    )
    
    
    year_fractions = []
    year_fractions.extend(t for t in yf_bafin if t not in year_fractions)
    year_fractions.extend(t for t in yf_current if t not in year_fractions)
    year_fractions.extend(t for t in yf_constant if t not in year_fractions)
    year_fractions = sorted(year_fractions)
    
    if useLowerBound:
        fig.add_trace(go.Scatter(x = year_fractions, y = lowerBound(year_fractions), name = 'Lower Bound', mode = plotly_scatter_mode_standard))
    
    if  useLowerBound:
        title_figure = 'Shock scenarios (with lower bound)'
    else :
        title_figure = 'Shock scenarios (without lower bound)'
    
    fig.update_layout(
        showlegend=True,
        xaxis = dict(title_text = "Expiry (in years)"),
        yaxis = dict(title_text = "Interest rate (in basis points)"),
        title={
            'text': title_figure,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'}
    )

    fig.show()

In [ ]:
# Use widgets to make the plot interactive

combobox_scenario = widgets.Combobox(
    # value='Flatten',
    placeholder='Choose Scenario',
    options=['ParallelUp', 'ParallelDown', 'ShortUp', 'ShortDown', 'LongUp', 'LongDown', 'Flatten', 'Steepen'],
    description='Scenario:',
    ensure_option=True,
    disabled=False
)

checkbox_lower_bound = widgets.Checkbox(
    value=False,
    description='Use lower bound',
    disabled=False,
    indent=True
)

widgets.interact(
    showPlot,
    shockScenario = combobox_scenario,
    shockParams = shockparams_standard,
    useLowerBound = checkbox_lower_bound
);

# TODO

- XXXX